Import of libraries

In [3]:
import folium
from folium.plugins import BeautifyIcon
import shapely
import pandas as pd
import openrouteservice as ors
from IPython.core.interactiveshell import InteractiveShell

load data, setting arbitrary amount for every point (1 in this case), view of the df

In [4]:
deliveries_data = pd.read_csv(r'path\to\the\file\punti_rm.csv')
deliveries_data['Needed_Amount'] =1
deliveries_data.drop(['id'], axis = 1, inplace = True) 
deliveries_data.head()


,x,y,Needed_Amount
0,12.578678,41.941710,1
1,12.446832,41.908528,1
2,12.520630,41.876203,1
3,12.521144,41.875115,1
4,12.520972,41.875457,1


Run only for visualization

In [5]:
# First define the map centered around Beira
m = folium.Map(location=[ 41.8719643, 12.4737054], tiles='cartodbpositron', zoom_start=8)    




# Plot the locations on the map with more info in the ToolTip
for location in deliveries_data.itertuples():
    tooltip = folium.map.Tooltip("<h4><b>ID {}</b></p><p>Supplies needed: <b>{}</b></p>".format(
        location.Index, location.Needed_Amount
    ))
    
    folium.Marker(
        location=[location.y, location.x],
        tooltip=tooltip,
        icon=BeautifyIcon(
            icon_shape='marker',
            number=int(location.Index),
            spin=True,
            text_color='red',
            background_color="#FFF",
            inner_icon_style="font-size:12px;padding-top:-5px;"
        )
    ).add_to(m)      
    

depot = [41.8719643, 12.4737054]

folium.Marker(
    location=depot,
    icon=folium.Icon(color="green", icon="bus", prefix='fa'),
    setZIndexOffset=1000
).add_to(m)

m

- vehicles start/end address: random point in Rome (depot = [41.8719643, 12.4737054])
- vehicle capacity: 300 (arbitrary)
- vehicle operational times: 08:00 - 20:00
- service location: delivery location -> pick up in my case!
- service time windows: individual delivery location's time window
- service amount: individual delivery location's needs

All the infos in [Vroom](https://github.com/VROOM-Project/vroom) and [ors](https://openrouteservice.org/disaster-optimization/)

In [19]:
# Define the vehicles
# https://openrouteservice-py.readthedocs.io/en/latest/openrouteservice.html#openrouteservice.optimization.Vehicle
vehicles = list()
for idx in range(3):
    vehicles.append(
        ors.optimization.Vehicle(
            id=idx, 
            start=list(reversed(depot)),
            #end=list(reversed(depot)),
            capacity=[300],
            time_window=[1553241600, 1553284800]  # Fri 8-20:00, expressed in POSIX timestamp
        )
    )

In [10]:
#open_from= 2019-03-22 08:00:00
#open_from.timestamp()

In [20]:
# Next define the delivery stations
# https://openrouteservice-py.readthedocs.io/en/latest/openrouteservice.html#openrouteservice.optimization.Job
deliveries = list()

open_from= "2019-03-22 08:00:00"

for delivery in deliveries_data.itertuples():
    deliveries.append(
        ors.optimization.Job(
            id=delivery.Index,
            location=[delivery.x, delivery.y],
            service=120,  # Assume 2 minutes at each site
            amount=[delivery.Needed_Amount],
            #time_windows=[[
            #    int(delivery.Open_From.timestamp()),  # VROOM expects UNIX timestamp
            #    int(delivery.Open_To.timestamp())
            #]]
        )
    )

Now the analysis

In [21]:
ors_client = ors.Client(key='yout key')  # Get an API key from https://openrouteservice.org/dev/#/signup
result = ors_client.optimization(
    jobs=deliveries,
    vehicles=vehicles,
    geometry=True
)
##############################################################################################################################
#Visualization only
# Add the output to the map
for color, route in zip(['green', 'red', 'blue'], result['routes']):
    decoded=ors.convert.decode_polyline(route['geometry'])  # Route geometry is encoded
    gj = folium.GeoJson(
        name='Vehicle {}'.format(route['vehicle']),
        data={"type": "FeatureCollection", "features": [{"type": "Feature", 
                                                         "geometry": decoded,
                                                         "properties": {"color": color}
                                                        }]},
        style_function=lambda x: {"color": x['properties']['color']}
    )
    gj.add_child(folium.Tooltip(
        """<h4>Vehicle {vehicle}</h4>
        <b>Distance</b> {distance} m <br>
        <b>Duration</b> {duration} secs
        """.format(**route)
    ))
    gj.add_to(m)

folium.LayerControl().add_to(m)
m

In [22]:
# Only extract relevant fields from the response
extract_fields = ['distance', 'amount', 'duration']
data = [{key: route[key] for key in extract_fields} for route in result['routes']]

vehicles_df = pd.DataFrame(data)
vehicles_df.index.name = 'vehicle'
vehicles_df

,distance,amount,duration
vehicle,,,
0,8951,[4],1340
1,55429,[17],5981
2,10841,[5],1464


In [23]:
# Create a list to display the schedule for all vehicles
stations = list()
for route in result['routes']:
    vehicle = list()
    for step in route["steps"]:
        vehicle.append(
            [
                step.get("job", "Depot"),  # Station ID
                step["arrival"],  # Arrival time
                step["arrival"] + step.get("service", 0),  # Departure time
                
            ]
        )
    stations.append(vehicle)

### Vehicle 0

Now we can look at each individual vehicle's timetable, for example for vehicle 0 (the same can be done for other 2 vehicles):

In [13]:
df_stations_0 = pd.DataFrame(stations[0], columns=["Station ID", "Arrival", "Departure"])
df_stations_0['Arrival'] = pd.to_datetime(df_stations_0['Arrival'], unit='s')
df_stations_0['Departure'] = pd.to_datetime(df_stations_0['Departure'], unit='s')
df_stations_0

,Station ID,Arrival,Departure
0,Depot,2019-03-22 08:00:00,2019-03-22 08:00:00
1,11,2019-03-22 08:09:45,2019-03-22 08:11:45
2,14,2019-03-22 08:19:25,2019-03-22 08:21:25
3,1,2019-03-22 08:24:23,2019-03-22 08:26:23
4,7,2019-03-22 08:28:20,2019-03-22 08:30:20
